In [ ]:
# test text, objectives, artifacts.
!pip install dotenv PyMuPDF


1699.56s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


KeyboardInterrupt: 

In [ ]:
# import stuff
from __future__ import annotations
import argparse, hashlib, json, os, sys
from datetime import datetime, UTC
from typing import List

from dotenv import load_dotenv
from openai import OpenAI
from pydantic import BaseModel, Field, ValidationError



load_dotenv()
client = OpenAI()
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o")

gpt-4o


In [ ]:
# artifact structure
# ── LLM function spec (updated) ─────────────────────────────────────────--
ART_FUNC = {
    "type": "function",
    "function": {
        "name": "artifact_generator",
        "description": "Generate artifact based on the following objective; machine, material, software, theory, workflow, principle, etc. \
            Include artifact name, description, inputs and outputs, and the laws that the artifact operates under.",
        "parameters": {
            "type": "object",
            "properties": {
                "artifacts": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "required": [
                            "name", "description", "inputs", "outputs", "laws"
                        ],
                        "properties": {
                            "name":  {"type": "string"},
                            "description": {"type": "string"},
                            "inputs": {"type": "string"},
                            "outputs": {"type": "string"},
                            "laws": {"type":"string"}
                        }
                    }
                }
            },
            "required": ["artifacts"]
        }
    }
}

In [ ]:
SYS_PROMPT = (
    "You are a design engineer. Using ONLY the text and the objective, "
    "propose up to {k} concrete artifacts. Each artifact must include:\n"
    "• tool_anchor (machine, material, software, theory, workflow, principle, etc.)\n"
    "• a description of the artifact\n"
    "• the inputs and outputs of the artifact\n"
    "• the laws which the artifact operates under.\n"
    #"• how this artifact achieves the objective.\n"
    # "Return JSON via function schema." 
    )

In [ ]:
import fitz  # PyMuPDF

library = "/Users/b/fantasiagenesis/crayon/library"

# List all files in the library (sorted for consistent ordering)
books = sorted(os.listdir(library))

# Select the second book (index 1 because indexing starts at 0)
library_text = {}
for book in books:
    book_path = os.path.join(library, book)
    # Read the content of the second book
    # Read text from the PDF
    with fitz.open(book_path) as doc:
        book_text = ""
        for page in doc:
            book_text += page.get_text()

    book_title = book.removesuffix(".pdf")
    library_text[book_title] = book_text

for text in library_text.keys():
    print(text)

Electricity_and_Magnetism_-_Purcell-3rd Edition-2
Gene-Cloning-and-DNA-Analysis


In [ ]:
book_section = book_text[0:5000]
#print(book_section)

src = {}
src["title"] = "Gene Cloning and DNA Analysis"
src["text"] = book_section

In [ ]:
obj = obj_existing
sys_msg = SYS_PROMPT.format(k=3)
print(sys_msg)
src = get_source(book_text, "Gene Cloning and DNA Analysis")
user_msg = f"OBJECTIVE:\n{obj}\n\nTITLE:\n{src["title"]}\nTEXT:\n{src["text"]}"
print(user_msg)

You are a design engineer. Using ONLY the text and the objective, propose up to 3 concrete artifacts. Each artifact must include:
• tool_anchor (machine, material, software, theory, workflow, principle, etc.)
• a description of the artifact
• the inputs and outputs of the artifact
• the laws which the artifact operates under.

OBJECTIVE:
What are the artifacts that exist/are in use based on the following text?

TITLE:
Gene Cloning and DNA Analysis
TEXT:
 Comparison between a typical glycosylation
structure found on a...
Figure 14.18 Three promoters frequently used in expression
factors for micro...
Figure 14.19 Crystalline inclusion bodies in the nuclei of
insect cells infe...
Figure 14.20 Transfer of the nucleus from a transgenic
somatic cell to an oo...
Figure 14.21 Recombinant protein production in the milk of a
transgenic shee...
Chapter 15
Figure 15.1 The structure of the insulin molecule and a
summary of its synth...
Figure 15.2 The synthesis of recombinant insulin from
artificia

In [ ]:
resp = client.chat.completions.create(
        model=OPENAI_MODEL,
        messages=[{"role":"system","content":sys_msg}, {"role":"user","content":user_msg}],
        tools=[ART_FUNC], tool_choice="auto", temperature=0.5
)


In [ ]:
import json
args_raw = resp.choices[0].message.tool_calls[0].function.arguments
raws = json.loads(args_raw)["artifacts"]

print(raws)

[{'name': 'Recombinant DNA Molecule', 'description': 'A recombinant DNA molecule is created by inserting a fragment of DNA, which contains a gene of interest, into a circular DNA molecule called a vector. This recombinant molecule is then introduced into a host cell, where it can replicate and produce multiple copies of the gene.', 'inputs': 'A fragment of DNA containing the gene of interest and a vector molecule.', 'outputs': 'A recombinant DNA molecule that can replicate within a host cell, producing multiple copies of the gene.', 'laws': 'Operates under the principles of molecular biology and genetic engineering, specifically the laws governing DNA replication and gene expression within host cells.'}, {'name': 'Polymerase Chain Reaction (PCR)', 'description': 'PCR is a technique used to amplify a specific segment of DNA, generating millions of copies of a particular DNA sequence. It involves repeated cycles of denaturation, annealing, and extension, carried out in a thermal cycler.'

In [232]:
def print_obj(obj, i=1, params=["description", "inputs", "outputs", "laws"]):
    print(f"{i} - ")
    for param in params:
        print(f"{param}: {obj[param]}")
    

def parse_func_resp(resp, obj_name="artifacts", params=["description", "inputs", "outputs", "laws"]):
    args_raw = resp.choices[0].message.tool_calls[0].function.arguments
    raws = json.loads(args_raw)[obj_name]

    obj_lst = []
    for i, obj in enumerate(raws, 1):
        print_obj(obj, i, params)
        obj_lst.append(obj)
    
    return obj_lst

In [ ]:
# be free of objectives. be free of money. just play. 
import random
def get_source(text, title, n=20000):
    n = 20000  # length of section in characters
    max_index = len(text) - n

    # Ensure we don't go out of bounds
    if max_index <= 0:
        book_section = text  # fallback: use the whole text
    else:
        index = random.randint(0, max_index)
        book_section = text[index : index + n]

    src = {}
    src["title"] = title
    src["text"] = book_section

    return src



In [ ]:
obj = obj_existing
sys_msg = SYS_PROMPT.format(k=3)
user_msg = (
    f"OBJECTIVE:\n{obj}\n\n"
    f"TITLE:\n{src['title']}\n"
    f"TEXT:\n{src['text']}"
)
print(sys_msg)
print(user_msg)


You are a design engineer. Using ONLY the text and the objective, propose up to 3 concrete artifacts. Each artifact must include:
• tool_anchor (machine, material, software, theory, workflow, principle, etc.)
• a description of the artifact
• the inputs and outputs of the artifact
• the laws which the artifact operates under.

OBJECTIVE:
What are the artifacts that exist/are in use based on the following text?

TITLE:
Electricity_and_Magnetism_-_Purcell-3rd Edition-2
TEXT:
isfy Eq. (9.18). Where is
this energy when it is traveling? How is it deposited in matter when it
arrives?
In the case of a static electric ﬁeld, such as the ﬁeld between the
plates of a charged capacitor, we found that the total energy of the system
could be calculated by attributing to every volume element dv an amount
of energy (ϵ0E2/2) dv and adding it all up. Look back at Eq. (1.53). Like-
wise, the energy invested in the creation of a magnetic ﬁeld could be cal-
culated by assuming that every volume element dv 

In [ ]:
library_artifacts = {}

for title, text in library_text.items():                                    # underscore = “throw-away” loop var
    for _ in range(10): 
        src = get_source(text, title)                  # ➜ {'title': ..., 'text': ...}

        sys_msg = SYS_PROMPT.format(k=3)
        user_msg = (
            f"OBJECTIVE:\n{obj}\n\n"
            f"TITLE:\n{src['title']}\n"
            f"TEXT:\n{src['text']}"
        )

        resp = client.chat.completions.create(
            model=OPENAI_MODEL,
            messages=[
                {"role": "system", "content": sys_msg},
                {"role": "user",   "content": user_msg},
            ],
            tools=[ART_FUNC],
            tool_choice="auto",
            temperature=0.5,
        )

        arts = parse_func_resp(resp)                   # ➜ list of artifacts
        library_artifacts.setdefault(title, []).extend(arts)


    

    

1. Uniqueness Theorem Proof Workflow
   Description: A structured workflow to prove the uniqueness theorem in electrostatics, which states that a solution for a given set of conductors with specified potentials is unique. This proof involves demonstrating that any difference between two solutions must satisfy Laplace's equation and be zero everywhere, ensuring the uniqueness of the solution.
   Inputs:      Assumptions: a solution φ(x, y, z) exists for given conductors with potentials φk, boundary conditions, Laplace's equation.
   Outputs:     Proof that the solution is unique, i.e., φ = ψ everywhere.
   Laws:        Laplace's equation, linearity of differential equations, boundary conditions, Gauss's law.

2. Electric Field Shielding Principle
   Description: A principle explaining the absence of electric fields inside a hollow conductor, which is used for electrical shielding. It states that the electric field inside a hollow conductor is zero if the space is empty of charge.
   Inp

In [ ]:
ri = random.randint(0, len(artifacts))
artifact = artifacts[0]
print_art(artifact)

1. Next Generation Sequencing Machines
   Description: Modern machines capable of sequencing both ends of a DNA fragment to obtain paired end reads, crucial for genome assembly without relying on chain termination sequencing.
   Inputs:      DNA fragments of various lengths (e.g., 150 bp, 500 bp, 2 kb, 5 kb, 10 kb).
   Outputs:     Paired end reads for genome assembly.
   Laws:        Operates under the principles of DNA sequencing and molecular biology, utilizing technologies that allow for high-throughput sequencing and data analysis.



In [ ]:
# what are the set of artifacts? What can you do with this artifact? What are the things that you build with this artifact? What are the relationships that you can build with other artifacts? 
# what are the stories? What are the situtaions? What can you build with this set of artifacts to impact the situation? 

In [ ]:
# so many colors. 

In [ ]:

# read the entire library
from pathlib import Path
from typing import Dict
from pathlib import Path
from typing import Dict, Union
import fitz              # PyMuPDF
import pytesseract
from PIL import Image
import io
from ebooklib import epub, ITEM_DOCUMENT
from bs4 import BeautifulSoup
from pathlib import Path
import pypandoc



from PIL import Image

Image.MAX_IMAGE_PIXELS = None  # disables the warning



def list_files_in_directory(
    dir_path: str | Path,
    *,
    recursive: bool = False
) -> Dict[Path, None]:
    """
    Return a dictionary {file_path: None} for all files in `dir_path`.
    Initially all values are None.
    """
    base = Path(dir_path).expanduser().resolve()
    if not base.is_dir():
        raise NotADirectoryError(f"{base} is not a directory.")

    pattern = "**/*" if recursive else "*"
    files = (p for p in base.glob(pattern) if p.is_file())
    return {p: None for p in files}

def extract_text_from_pdf(pdf_path: Path) -> str:
    """Try to extract text using PyMuPDF. Return empty string if no text found."""
    text = ""
    try:
        doc = fitz.open(pdf_path)
        text = "\n".join(page.get_text() for page in doc)
    except Exception as e:
        print(f"⚠️ Error reading PDF {pdf_path.name}: {e}")
    return text.strip()

def extract_text_from_pdf_with_ocr(pdf_path: Path) -> str:
    """Fallback: use OCR if no text was extractable from PDF."""
    try:
        doc = fitz.open(pdf_path)
        ocr_text = []
        for page in doc:
            pix = page.get_pixmap(dpi=300)
            img = Image.open(io.BytesIO(pix.tobytes("png")))
            text = pytesseract.image_to_string(img)
            ocr_text.append(text)
        return "\n".join(ocr_text).strip()
    except Exception as e:
        print(f"⚠️ OCR failed on {pdf_path.name}: {e}")
        return ""

def extract_text_from_epub(epub_path: Path) -> str:
    try:
        book = epub.read_epub(str(epub_path))
    except Exception as e:
        print(f"⚠️ Failed to load EPUB {epub_path.name}: {e}")
        return ""

    texts = []

    for item in book.get_items():
        if item.get_type() == ITEM_DOCUMENT:  # ← use the imported constant
            soup = BeautifulSoup(item.get_content(), "html.parser")
            text = soup.get_text(separator=" ", strip=True)
            if text:
                texts.append(text)

    return "\n\n".join(texts).strip()

def read_rtf_file(path: Path) -> str:
    return pypandoc.convert_file(str(path), 'plain')
    
def read_library_files(
    files: list[Path],
    *,
    encoding: str = "utf-8"
) -> Dict[Path, str]:
    """
    Read content from .txt and .pdf files.
    PDF fallback uses OCR if no extractable text.
    """
    library: Dict[Path, str] = {}

    for path in files:
        if path.suffix.lower() == ".txt":
            try:
                library[path] = path.read_text(encoding=encoding)
            except Exception as e:
                print(f"⚠️ Skipping unreadable .txt file: {path.name} ({e})")

        elif path.suffix.lower() == ".pdf":
            print(f"📄 Reading PDF: {path.name}")
            text = extract_text_from_pdf(path)
            if not text:
                print(f"🧠 Trying OCR for {path.name} …")
                text = extract_text_from_pdf_with_ocr(path)
            if text:
                library[path] = text
            else:
                print(f"⚠️ No text could be extracted from {path.name}")

        elif path.suffix.lower() == ".epub":
            print(f"📚 Reading EPUB: {path.name}")
            text = extract_text_from_epub(path)
            if text:
                library[path] = text
            else:
                print(f"⚠️ No text extracted from {path.name}")

        elif path.suffix.lower() == ".rtf":
            print(f"📚 Reading EPUB: {path.name}")
            text = read_rtf_file(path)
            if text:
                library[path] = text
            else:
                print(f"⚠️ No text extracted from {path.name}")

    return library


In [ ]:
# 1. Get all files
library_dir = "/Users/b/fantasiagenesis/crayon/library/"
from pathlib import Path
files = list(Path(library_dir).rglob("*"))  # recursive

# 2. Load readable content
library = read_library_files(files[:70])

# 3. Check results
print(f"✓ Loaded {len(library)} readable files")
for path, text in list(library.items())[:5]:
    print(f"— {path.name} ({len(text):,} chars)")


📄 Reading PDF: hogan_chapter_7_handbook_on_the_economics_of_electricity_112619r.pdf
📄 Reading PDF: The Birds of East Africa- Kenya, Tanzania, Uganda, Rwanda, -- Terry Stevenson, John Fanshawe -- Princeton Field Guides, Princeton, New Jersey, Soho -- 9780691126654 -- 65438af214a8415b96bb9f23f49bbcbc -- Anna’s Archive.pdf
📄 Reading PDF: 978-981-16-5214-1.pdf
📄 Reading PDF: Electricity_and_Magnetism_-_Purcell-3rd Edition-2.pdf
📄 Reading PDF: watson-levin-2023-the-collective-intelligence-of-evolution-and-development.pdf
📄 Reading PDF: book.pdf
📄 Reading PDF: s11047-013-9398-1.pdf
📄 Reading PDF: annas-arch-f5d0d68bc440.pdf
📄 Reading PDF: PIIS2589004221000997.pdf
📄 Reading PDF: scirobotics.abf1571.pdf
📄 Reading PDF: PIIS0092867421002233-2.pdf
📄 Reading PDF: 1-s2.0-S0016508517361486-main.pdf
📄 Reading PDF: kcib-09-04-1192733.pdf
📄 Reading PDF: preview-9781136665493_A37885936.pdf
📄 Reading PDF: nihms120942.pdf
📄 Reading PDF: 1-s2.0-S0959440X15000111-main.pdf
📄 Reading PDF: annurev-bioeng-07111

In [ ]:
print(len(library))

69


In [ ]:
n_empty = sum(1 for text in library.values() if not text)

print(f"{n_empty} out of {len(library)} files are empty.")
for path, text in library.items():
    if not text or not text.strip():
        print("—", path.name)

1 out of 69 files are empty.
— Dr. Takenori Kobayashi-Impact of Battery Energy Storage System.txt


In [ ]:
library_artifacts = {}

for title, text in library.items():   # underscore = “throw-away” loop var
    if not text or not text.strip():
        pass
    else:                                 
        for _ in range(5): 
            src = get_source(text, title)                  # ➜ {'title': ..., 'text': ...}

            sys_msg = SYS_PROMPT.format(k=3)
            user_msg = (
                f"OBJECTIVE:\n{obj}\n\n"
                f"TITLE:\n{src['title']}\n"
                f"TEXT:\n{src['text']}"
            )

            resp = client.chat.completions.create(
                model=OPENAI_MODEL,
                messages=[
                    {"role": "system", "content": sys_msg},
                    {"role": "user",   "content": user_msg},
                ],
                tools=[ART_FUNC],
                tool_choice="auto",
                temperature=0.5,
            )

            arts = parse_func_resp(resp)                   # ➜ list of artifacts
            library_artifacts.setdefault(title, []).extend(arts)

1. Financial Transmission Rights (FTR) System
   Description: A system that provides a financial contract to hedge against the volatility of locational price differences in electricity markets. It allows market participants to manage long-term contracts between different locations on the grid by offering payments based on the price difference between two points.
   Inputs:      MW amount, direction of flow, locational prices (p_j, p_i)
   Outputs:     Payments for the price difference (p_j - p_i) for the designated MW amount.
   Laws:        Operates under the laws of supply and demand, competitive equilibrium, and principles of economic dispatch. It also adheres to the principles of transmission open access and non-discrimination.

2. Locational Marginal Pricing (LMP) Model
   Description: A pricing model used in electricity markets to determine the cost of electricity at different locations, accounting for energy cost, marginal losses, and congestion. It supports efficient market out

In [ ]:
# what can i do with this thing? What problems can I solve, tell story_inputs 
# story inputs and outputs
# then string together one story and another. 

# tell stories and string together stories...


In [ ]:
ri = random.randint(0, len(artifacts))
artifact_object = (list(library_artifacts.keys())[ri], list(library_artifacts.values())[ri])
artifact_path, artifact = artifact_object
artifact = artifact[0]
print(artifact_path)
print_art(artifact)

/Users/b/fantasiagenesis/crayon/library/s11047-013-9398-1.pdf
1. Programmable Self-Organization Systems
   Description: Systems designed to autonomously and reproducibly form large heterogeneous architectures without central planning, utilizing local interactions and decentralized control. These systems are based on the principles of morphogenetic engineering, which explores the design and control of complex systems capable of self-organization.
   Inputs:      Local interaction rules, decentralized control algorithms, environmental feedback.
   Outputs:     Emergent complex architectures, decentralized systems with desired functionalities.
   Laws:        Operates under the principles of decentralized control, emergent behavior, and self-organization as defined by morphogenetic engineering.



In [ ]:
SYS_PROMPT_ARTIFACT = (
    "You are a design engineer. Using ONLY the given artifact, "
    "propose up to {k} concrete scenarios in which this artifact is used."
    #"• how this artifact achieves the objective.\n"
    # "Return JSON via function schema." 
    )
sys_msg = SYS_PROMPT.format(k=3)
user_msg = (
    f"OBJECTIVE:\n{obj}\n\n"
    f"TITLE:\n{src['title']}\n"
    f"TEXT:\n{src['text']}"
)

In [ ]:
def artifact_as_string(artifact):
    return f"ARTIFACT:\n{artifact["name"]}\n\nDESCRIPTION:\n{artifact["description"]}\n\nINPUTS:\n{artifact["inputs"]}\n\nOUTPUTS:\n{artifact["outputs"]}\n\nLAWS:\n{artifact["laws"]}"

In [ ]:
SYS_PROMPT_ARTIFACT = "You are a brutally practical R&D advisor."

obj = "Propose up to {k} concrete scenarios in which the following artifact is used."

In [ ]:
sys_msg_objective = SYS_PROMPT_ARTIFACT
user_msg_objective = f"Artifact (one-liner): {{artifact_name}}"
    f"Key capability: {{artifact_description}}"
    "Interfaces & deps: software"
    "My envelope: Hardware: Computer, GPU, ...; Budget: $500; Time: 3 weekends; Target TRL: demo to classmates, not a publishable paper"
    "Goal grade: Beta tool" # Proof-of-concept | Beta tool | Production
    "Generate {k} build slices that:"
    "• each end in a tangible demo or dataset"
    "• escalate in scope"
    "• fit the envelope"
    "For each slice output:"
    "1. Hook – the visible win"
    "2. Required additions (parts, libs, datasets) with ball-park cost"
    "3. Step-by-step build plan (≤7 steps)"
    "4. Proof test & success metric"
    "5. How it exercises the artifact’s core capability"
    "6. Next slice it unlocks"

In [197]:
resp = client.chat.completions.create(
        model=OPENAI_MODEL,
        messages=[{"role":"system","content":sys_msg_objective}, {"role":"user","content":user_msg_objective}],
        #tools=[ART_FUNC], tool_choice="auto", 
        temperature=0.5
    )

print(f"ARTIFACT {ri}:\n{artifact_as_string(artifact)}\n\n{resp.choices[0].message.content}")

ARTIFACT 6:
ARTIFACT:
Programmable Self-Organization Systems

DESCRIPTION:
Systems designed to autonomously and reproducibly form large heterogeneous architectures without central planning, utilizing local interactions and decentralized control. These systems are based on the principles of morphogenetic engineering, which explores the design and control of complex systems capable of self-organization.

INPUTS:
Local interaction rules, decentralized control algorithms, environmental feedback.

OUTPUTS:
Emergent complex architectures, decentralized systems with desired functionalities.

LAWS:
Operates under the principles of decentralized control, emergent behavior, and self-organization as defined by morphogenetic engineering.

1. **Disaster Recovery and Reconstruction:**
   In the aftermath of a natural disaster, such as an earthquake or hurricane, programmable self-organization systems could be deployed to autonomously reconstruct essential infrastructure. These systems could use loca

In [198]:
for _ in range(50):
    ri = random.randint(0, len(artifacts))
    artifact_object = (list(library_artifacts.keys())[ri], list(library_artifacts.values())[ri])
    artifact_path, artifact = artifact_object
    artifact = artifact[0]

    user_msg_objective = f"OBJECTIVE:\n{obj}\n\nARTIFACT:\n{artifact_as_string(artifact)}"

    resp = client.chat.completions.create(
        model=OPENAI_MODEL,
        messages=[{"role":"system","content":sys_msg_objective}, {"role":"user","content":user_msg_objective}],
        #tools=[ART_FUNC], tool_choice="auto", 
        temperature=0.5
    )

    print(artifact_path)
    print(f"ARTIFACT {ri}:\n{artifact_as_string(artifact)}\n\n{resp.choices[0].message.content}")

    

    

    

/Users/b/fantasiagenesis/crayon/library/s11692-018-9448-9.pdf
ARTIFACT 22:
ARTIFACT:
Planarian Regenerative Model

DESCRIPTION:
A computational model simulating the regenerative processes of planaria, focusing on the role of neoblasts and their behavior in response to environmental changes.

INPUTS:
Initial conditions of neoblast distribution and genetic information, environmental parameters, and signaling cues.

OUTPUTS:
Simulated outcomes of planarian regeneration, including neoblast distribution, organ regrowth, and potential anomalies.

LAWS:
Operates under biological principles of regeneration, genetic variability, and bioelectric signaling pathways.

1. **Biomedical Research and Drug Testing**: Researchers can use the Planarian Regenerative Model to simulate how different drugs or genetic modifications affect planarian regeneration. By altering the inputs related to genetic information or environmental parameters, scientists can predict how these changes impact neoblast behavior 

In [201]:
for _ in range(5):
    ri = random.randint(0, len(artifacts))
    artifact_object = (list(library_artifacts.keys())[ri], list(library_artifacts.values())[ri])
    artifact_path, artifact = artifact_object
    artifact = artifact[0]

    print(artifact_path)
    print_art(artifact, ri)

    

/Users/b/fantasiagenesis/crayon/library/1-s2.0-S0016508517361486-main.pdf
11. CRISPR/Cas9 Mouse Model
   Description: A genetically engineered mouse model using CRISPR/Cas9 technology to induce FL-HCC by creating the Dnajb1–Prkaca fusion, mimicking human fibrolamellar hepatocellular carcinoma (FL-HCC).
   Inputs:      CRISPR/Cas9 components, mouse genome, guide RNAs targeting Dnajb1 and Prkaca genes.
   Outputs:     Mouse models exhibiting FL-HCC with Dnajb1–Prkaca fusion, oncogenic lesions in liver parenchyma.
   Laws:        Operates under genetic engineering principles, CRISPR/Cas9 genome editing mechanisms, Mendelian inheritance, and biological laws of tumorigenesis.

/Users/b/fantasiagenesis/crayon/library/annas-arch-f5d0d68bc440.pdf
7. Toyota's Multi-Product and Small Volume Production System
   Description: A revolutionary car production system developed by Toyota that focuses on producing multiple products in small volumes. This system is designed to maintain harmony between ma

In [ ]:

# imaging


# 11. CRISPR/Cas9 Mouse Model
data11p3 = "What datasets can you collect on CRISPR/Cas9 technology? FL-HCC? Dnajb1–Prkaca fusion? human fibrolamellar hepatocellular carcinoma?"
data11p4 = "What datasets can you collect on CRISPR/Cas9 components, mouse genome, guide RNAs targeting Dnajb1 and Prkaca genes?"
data11p5 = "What datasets can you collect on Mouse models exhibiting FL-HCC with Dnajb1–Prkaca fusion? oncogenic lesions in liver parenchyma?"
data11p6 = "What datasets can you collect on genetic engineering principles? CRISPR/Cas9 genome editing mechanisms? Mendelian inheritance? biological laws of tumorigenesis?"


# 7. Toyota's Multi-Product and Small Volume Production System
# A revolutionary car production system developed by Toyota that focuses on producing multiple products in small volumes. This system is designed to maintain harmony between mass production and mass distribution, adapting to global market needs.
data7p1 = "What datasets can you collect on Toyota's car production system? mass production? mass distribution? global market needs?"
data7p2 = "What datsets can you collect on Toyota Market demands? production resources? design specifications?"
data7p3 = "What datasets can you collect on car models? model production? volume production?"
data7p4 = "What datasets can you collect on the principles of lean manufacturing? on just-in-time production?"


# 21. Transfection and Virus Collection Workflow
# A process for transfecting HEK293T cells with a mixture and collecting virus from the media post-transfection.
data21p1 = "What datasets can you collect on HEK293T cells? on transfection of HEK293T cells? on virus collection?"
data21p2 = "What datasets can you collect on HEK293T cells? Opti-MEM? transfection mixture? DMEM? Pen/Strep?"
data21p1 = "What datasets can you collect Virus collection from media? viral storage?"

# 1. Bird Identification Guide
# A comprehensive field guide for identifying birds in East Africa, specifically in regions such as Kenya, Tanzania, Uganda, and Rwanda. The guide includes detailed descriptions of bird species, their physical characteristics, habitats, and vocalizations.
data1p1 = "What datsastes can you collect on birds in East Africa? What datasets can you collect on birds in Kenya? Tanzania? Uganda? Rwanda? What datasets can you collect on their physical characteristics, habitats, and vocalizations?"
data1p2 = "What datasets can you collect on observations of bird species in the field? their physical characteristics and vocalizations? What datasets can you collect on the identification of bird species based on observed characteristics? bird watchers? and researchers?"
data1p3 = "What datasets can you collect on the principles of ornithology and taxonomy? scientific classification and identification methods?"

# 18. Flywheel Energy Storage System
# A mechanical device that stores energy in the form of kinetic energy using a rotating flywheel. The system includes components like motors, generators, and various safety and efficiency measures to ensure optimal energy storage and retrieval.
data18p1 = "What datasets can you collect on energy storage using a rotating flywheel? system composition of flywheels: motor, generators? What dataset can you collect on safety and efficiency measures of flywheels? on optimization of energy storage and retrieval?"
data18p2 = "What datasets can you collect on Moments of inertia of flywheel? angular velocity of flywheels? material properties of flywheels? motor/generator specifications of flywheels?"
data18p3 = "What datasets can you collect on Stored kinetic energy of flywheels? rotational speed of flywheels? system efficiency of flywheels?"
data18p4 = "What datasets can you collect on Conservation of energy? Newton's laws of motion? principles of electromagnetism?"


In [207]:
SYSTEM_PROMPT_DATA = "You are a data-oriented R&D assistant.  \n" \
"When asked for a “Variable Scaffold” you must:\n" \
"  • Identify the measurable phenomena that expose the artifact in the real world.  \n" \
"  • For each phenomenon, produce exactly four fields:\n" \
"        {name, units, sampling, source_hint}.  \n" \
"  • Keep names snake_case; pick concrete units; make sampling explicit\n" \
"        (e.g. “hourly”, “per image”, “every 5 s event”).  \n" \
"  • Output as a markdown table, no prose.  \n" \
"  • Include 5–12 variables total.  \n" \
"  • If uncertain, mark the source_hint with “??”."

#user_msg_objective = f"ARTIFACT:\n{artifact_as_string(artifact)}"

In [245]:
SYSTEM_PROMPT_DATA = "You are a data-oriented R&D assistant.  \n" \
"When asked for a “Variable Scaffold” you must:\n" \
"  • Identify the measurable phenomena that expose the artifact in the real world.  \n" \
"  • For each phenomenon, produce exactly four fields:\n" \
"        {name, units, sampling, source_hint}.  \n" \
"  • Keep names snake_case; pick concrete units; make sampling explicit\n" \
"        (e.g. “hourly”, “per image”, “every 5 s event”).  \n" \
"  • No prose.  \n" \
"  • Include 5–12 variables total.  \n" \
"  • If uncertain, mark the source_hint with '??'."

DATA_FUNC = {
    "type": "function",
    "function": {
        "name": "data_collection",
        "description": "When asked for a “Variable Scaffold” you must:\n" \
                        "  • Identify the measurable phenomena that expose the artifact in the real world.  \n" \
                        "  • For each phenomenon, produce exactly four fields:\n" \
                        "        {name, units, sampling, source_hint}.  \n" \
                        "  • Keep names snake_case; pick concrete units; make sampling explicit\n" \
                        "        (e.g. “hourly”, “per image”, “every 5 s event”).  \n" \
                        "  • No prose.  \n" \
                        "  • Include 5–12 variables total.  \n" \
                        "  • If uncertain, mark the source_hint with '??'.",
        "parameters": {
            "type": "object",
            "properties": {
                "data_variables": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "required": [
                            "name", "units", "sampling", "source_hint"
                        ],
                        "properties": {
                            "name":  {"type": "string"},
                            "units": {"type": "string"},
                            "sampling": {"type": "string"},
                            "source_hint": {"type": "string"}
                        }
                    }
                }
            },
            "required": ["data_variables"]
        }
    }
}

In [246]:
print(DATA_FUNC["function"]["description"])

When asked for a “Variable Scaffold” you must:
  • Identify the measurable phenomena that expose the artifact in the real world.  
  • For each phenomenon, produce exactly four fields:
        {name, units, sampling, source_hint}.  
  • Keep names snake_case; pick concrete units; make sampling explicit
        (e.g. “hourly”, “per image”, “every 5 s event”).  
  • No prose.  
  • Include 5–12 variables total.  
  • If uncertain, mark the source_hint with '??'.


In [247]:
data_variables = {}

for i in [11, 7, 21, 1, 18]:
    artifact_object = (list(library_artifacts.keys())[i], list(library_artifacts.values())[i])
    artifact_path, artifact = artifact_object
    artifact = artifact[0]

    sys_msg_objective = SYSTEM_PROMPT_DATA
    user_msg_objective = f"ARTIFACT:\n{artifact_as_string(artifact)}"

    resp = client.chat.completions.create(
        model=OPENAI_MODEL,
        messages=[{"role":"system","content":sys_msg_objective}, {"role":"user","content":user_msg_objective}],
        tools=[DATA_FUNC], tool_choice="auto", 
        temperature=0.5
    )

    objs = parse_func_resp(resp, "data_variables", ["name", "units", "sampling", "source_hint"])

    data_variables[str(i)] = objs



    

1 - 
name: fusion_gene_presence
units: binary
sampling: per mouse
source_hint: genetic assay
2 - 
name: tumor_size
units: mm^3
sampling: monthly
source_hint: MRI scan
3 - 
name: survival_rate
units: percentage
sampling: annual
source_hint: longitudinal study
4 - 
name: liver_function_markers
units: IU/L
sampling: quarterly
source_hint: blood test
5 - 
name: weight_change
units: grams
sampling: weekly
source_hint: physical examination
6 - 
name: guide_rna_efficiency
units: percentage
sampling: per experiment
source_hint: genetic assay
7 - 
name: mutation_rate
units: mutations per genome
sampling: per mouse
source_hint: genomic sequencing
8 - 
name: cas9_activity
units: nM/s
sampling: per experiment
source_hint: enzymatic assay
9 - 
name: health_score
units: index
sampling: monthly
source_hint: veterinary assessment
1 - 
name: production_volume
units: units/day
sampling: daily
source_hint: production_reports
2 - 
name: product_variants
units: count
sampling: per production cycle
source_h

1 - 
name: species_name
units: text
sampling: per observation
source_hint: field guide
2 - 
name: physical_characteristics
units: text
sampling: per observation
source_hint: field guide
3 - 
name: habitat_description
units: text
sampling: per observation
source_hint: field guide
4 - 
name: vocalization_description
units: text
sampling: per observation
source_hint: field guide
5 - 
name: geographical_location
units: coordinates
sampling: per observation
source_hint: GPS
6 - 
name: observation_date
units: date
sampling: per observation
source_hint: field notes
7 - 
name: observer_name
units: text
sampling: per observation
source_hint: field notes
8 - 
name: weather_conditions
units: text
sampling: per observation
source_hint: field notes
9 - 
name: time_of_day
units: time
sampling: per observation
source_hint: field notes
1 - 
name: angular_velocity
units: radians_per_second
sampling: every_5_s
source_hint: sensor
2 - 
name: stored_kinetic_energy
units: joules
sampling: every_5_s
source_

In [208]:
print(SYSTEM_PROMPT_DATA)

You are a data-oriented R&D assistant.  
When asked for a “Variable Scaffold” you must:
  • Identify the measurable phenomena that expose the artifact in the real world.  
  • For each phenomenon, produce exactly four fields:
        {name, units, sampling, source_hint}.  
  • Keep names snake_case; pick concrete units; make sampling explicit
        (e.g. “hourly”, “per image”, “every 5 s event”).  
  • Output as a markdown table, no prose.  
  • Include 5–12 variables total.  
  • If uncertain, mark the source_hint with “??”.


In [228]:
print(len(data_variables))

5


In [237]:
def data_variable_as_string(data_variable):
    return f"DATA VARIABLE:\n{data_variable["name"]}\n\nUNITS:\n{data_variable["units"]}\n\nSAMPLING:\n{data_variable["sampling"]}\n\nSOURCE_HINT:\n{data_variable["source_hint"]}"


In [248]:
i = 11
artifact_object = (list(library_artifacts.keys())[i], list(library_artifacts.values())[i])
artifact_path, artifact = artifact_object
artifact = artifact[0]

print(artifact_as_string(artifact), "\n\n")
print(data_variable_as_string(data_variables[str(i)][0]))



ARTIFACT:
CRISPR/Cas9 Mouse Model

DESCRIPTION:
A genetically engineered mouse model using CRISPR/Cas9 technology to induce FL-HCC by creating the Dnajb1–Prkaca fusion, mimicking human fibrolamellar hepatocellular carcinoma (FL-HCC).

INPUTS:
CRISPR/Cas9 components, mouse genome, guide RNAs targeting Dnajb1 and Prkaca genes.

OUTPUTS:
Mouse models exhibiting FL-HCC with Dnajb1–Prkaca fusion, oncogenic lesions in liver parenchyma.

LAWS:
Operates under genetic engineering principles, CRISPR/Cas9 genome editing mechanisms, Mendelian inheritance, and biological laws of tumorigenesis. 


DATA VARIABLE:
fusion_gene_presence

UNITS:
binary

SAMPLING:
per mouse

SOURCE_HINT:
genetic assay


In [254]:
SYSTEM_PROMPT_SOURCE = "You are a 'Source-Map bot'.\n" \
"Given a Variable Scaffold, you must—for **each variable**—\n" \
"  1. Search the web for at least two real data feeds.  \n" \
"  2. Fill the table with the columns  \n" \
"        source_id · access_method · license · update_freq  \n" \
"  3. Keep `source_id` short, snake-case, unique.  \n" \
"  4. Cite every web source found.  \n" \
"  5. If a field is unclear, write '??'.\n\n" \
"Return a markdown table only—no prose." 

In [250]:
print(SYS_PROMPT_SOURCE)

You are a 'Source-Map bot'.
Given a Variable Scaffold, you must—for **each variable**—
  1. Search the web for at least two real data feeds.  
  2. Fill the table with the columns  
        source_id · access_method · license · update_freq  
  3. Keep `source_id` short, snake-case, unique.  
  4. Cite every web source found.  
  5. If a field is unclear, write '??'.

Return a markdown table only—no prose.


In [251]:
data_sources = {}

for i in [11, 7, 21, 1, 18]:
    artifact_object = (list(library_artifacts.keys())[i], list(library_artifacts.values())[i])
    artifact_path, artifact = artifact_object
    artifact = artifact[0]

    sys_msg_objective = SYSTEM_PROMPT_SOURCE
    for data_variable in data_variables[str(i)]:
        user_msg_objective = f"ARTIFACT:\n{artifact_as_string(artifact)}\n\nVARIABLE:\n{data_variable_as_string(data_variable)}"

        resp = client.chat.completions.create(
            model=OPENAI_MODEL,
            messages=[{"role":"system","content":sys_msg_objective}, {"role":"user","content":user_msg_objective}],
            #tools=[DATA_FUNC], tool_choice="auto", 
            temperature=0.5
        )

        objs = parse_func_resp(resp, "data_sources", ["name", "units", "sampling", "source_hint"])

        data_sources[str(i)] = objs


NameError: name 'SYSTEM_PROMPT_SOURCE' is not defined

In [265]:
i = 11
artifact_object = (list(library_artifacts.keys())[i], list(library_artifacts.values())[i])
artifact_path, artifact = artifact_object
artifact = artifact[0]

SYSTEM_PROMPT_SOURCE = "You are a 'Source-Map bot'.\n" \
"Given a Variable Scaffold, you must—for **each variable**—\n" \
"  1. Search the web for at least two real data feeds.  \n" \
"  2. Fill the table with the columns  \n" \
"        source_id · access_method · license · update_freq  \n" \
"  3. Keep `source_id` short, snake-case, unique.  \n" \
"  4. Cite every web source found.  \n" \
"  5. If a field is unclear, write '??'.\n\n" \
"Return a markdown table only—no prose." 

sys_msg_objective = SYSTEM_PROMPT_SOURCE


for data_variable in data_variables[str(i)]:

    user_msg_objective = f"ARTIFACT:\n{artifact_as_string(artifact)}\n\nVARIABLE:\n{data_variable_as_string(data_variable)}"


    response = client.responses.create(
        model=OPENAI_MODEL, #"gpt-4.1-mini",  # or another supported model
        input = f"{sys_msg_objective}\n\n{user_msg_objective}",
        #messages=[{"role":"system","content":sys_msg_objective}, {"role":"user","content":user_msg_objective}],
        tools=[
            {
                "type": "web_search"
            }
        ],
        #temperature=0.5
    )

    print(artifact_as_string(artifact))
    print(data_variable_as_string(data_variable))
    print_web_search(response)

ARTIFACT:
CRISPR/Cas9 Mouse Model

DESCRIPTION:
A genetically engineered mouse model using CRISPR/Cas9 technology to induce FL-HCC by creating the Dnajb1–Prkaca fusion, mimicking human fibrolamellar hepatocellular carcinoma (FL-HCC).

INPUTS:
CRISPR/Cas9 components, mouse genome, guide RNAs targeting Dnajb1 and Prkaca genes.

OUTPUTS:
Mouse models exhibiting FL-HCC with Dnajb1–Prkaca fusion, oncogenic lesions in liver parenchyma.

LAWS:
Operates under genetic engineering principles, CRISPR/Cas9 genome editing mechanisms, Mendelian inheritance, and biological laws of tumorigenesis.
DATA VARIABLE:
fusion_gene_presence

UNITS:
binary

SAMPLING:
per mouse

SOURCE_HINT:
genetic assay
| source_id               | access_method | license | update_freq |
|-------------------------|---------------|---------|-------------|
| gen_assay_database      | API/Web       | ??      | Daily       |
| crispertech_journal     | Web           | Open    | Quarterly   |

References:
- [Genetic Assays Database]

In [262]:
# imagine you already have the Response object in `resp`
# 1️⃣ turn it into ordinary data
def print_web_search(response):
    blob = response.model_dump()          # or `.model_dump_json()` for a JSON string
    # older client versions: use `resp.dict()` / `resp.json()`

    # 2️⃣ grab the assistant's textual answer
    assistant_text = next(
        part["text"]
        for item in blob["output"]
        if item["type"] == "message"
        for part in item["content"]
        if part["type"] == "output_text"
    )

    # 3️⃣ grab the URLs cited
    citations = [
        ann["url"]
        for item in blob["output"]
        if item["type"] == "message"
        for part in item["content"]
        if part["type"] == "output_text"
        for ann in part.get("annotations", [])
        if ann["type"] == "url_citation"
    ]

    print(assistant_text)
    print(citations)
